In [1]:
import pandas as pd
import numpy as np
# import matplotlib
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn import datasets
import collections
import datetime
from tqdm import tqdm_notebook as tqdm
# %matplotlib inline 
# sns.set(color_codes=True)
# from numpy import median
import re
# from translate import Translator
import pickle

# Comparison Data

In [81]:
from peewee import Model, TextField, BooleanField
from playhouse.postgres_ext import PostgresqlExtDatabase, JSONField , IntegerField
from playhouse.shortcuts import model_to_dict


db_name = 'bitech'
db_login = 'rinat'
db_password = 'h786t8f8'
db_host = 'sw.neafiol.site'
db_port = 5432

db = PostgresqlExtDatabase(db_name, user=db_login, password=db_password,
                           host=db_host, port=db_port)

class SearchResult(Model):
    orig_name = TextField(unique=True)
    name = TextField(default="")
    url = TextField(default="")
    options = JSONField(default=[])
    specifications = JSONField(default={})
    done = BooleanField(default=False)

    class Meta:
        database = db
        
class MergeData(Model):
    catalog_name = TextField(default="")
    item_name = TextField(default="")
    type = TextField(default="")
    fatness = TextField(default="")
    sort = TextField(default="")
    fasofka = TextField(default="")
    milk_type = TextField(default="")
    cheese_type = TextField(default="")
    weight = TextField(default="")
    yamarket = TextField(default="")
    score = TextField()
    mark_type = IntegerField()

    class Meta:
        database = db

In [80]:
# MergeData.drop_table()
# MergeData.create_table()

In [82]:
MergeData.select().count()

49436

In [5]:
def clear_name(name):

    NUM = r"\d+[,\.]?\d*"

    def to_float(string):
        return float(re.search(NUM, string).group(0).replace(",", "."))

    data = []
    p = re.search(f"{NUM}\s*кг", name)
    if p:
        data.append(to_float(p.group(0)) * 1000)
        name = name.replace(p.group(0), "")

    p = re.search(f"{NUM}\s*г", name)
    if p:
        data.append(to_float(p.group(0)))
        name = name.replace(p.group(0), "")

    p = re.search(f"{NUM}\s*%", name)
    if p:
        data.append(to_float(p.group(0)))
        name = name.replace(p.group(0), "")

    name = re.sub(r"[^а-яa-z\-]", " ", name.lower())
    name = re.sub(r"\s+", " ", name.lower().replace("упаковка","").replace("НЕИДЕНТИФИЦИРУЕМАЯ МАРКА",""))
    return data, name

In [6]:
data = pd.read_csv("data/Справочник.csv",sep=';')
data = data[data["CategoryName"]=="СЫРЫ"]
data2 = pd.read_csv("data/Справочник 2.csv",sep=';')
with open("sresult.pk","rb") as f:
    sresult = pickle.load(f)

In [7]:
datar = data.to_dict("records")
data2r = data2.to_dict("records")

name_to_kw = {}

def to_set(text):
    _ , text = clear_name(text)
    return set(filter(lambda x: len(x) > 2, text.split(" ")))

for d in datar:
    name_to_kw[d["UniqueName"]] = to_set(" ".join([v for v in d.values() if isinstance(v,str)]))


for d in data2r:
    name_to_kw[d["Полное наименование товара"]] = to_set(" ".join([v for v in d.values() if isinstance(v,str)]))

    

In [8]:
name_to_kw['СЫР ТВЕРДЫЙ БРУСОК КАЧОТТА FORMAGGI 0,12 кг трюфель']

{'formaggi', 'ашан', 'брусок', 'качотта', 'сыр', 'сыры', 'твердый', 'трюфель'}

In [9]:
names = list(data["UniqueName"])
names2 = list(data2["Полное наименование товара"])

# names = list(data2["Полное наименование товара"])


# n = len(names)
# for i in tqdm(range(0,n,5000)):
#      SearchResult.insert_many([{"orig_name":clear_name(n)[1]} for n in names[i:i+5000]]).on_conflict_ignore().execute()
    

In [10]:
dctresult = {s["orig_name"]:s for s in sresult}

In [11]:
options = []
for name in names2:
    nums, orig_name = clear_name(name)
    options.append({
        "nums":nums,
        "name":orig_name,
        "orig":name,
        **dctresult[orig_name]
    })
    

In [33]:
similars_items = {}

In [10]:
for name in tqdm(names):
    nums, orig_name = clear_name(name)
    item = dctresult[orig_name]
    similars_items[name] = {
        "orig":name,
        "nums":nums,
        "item":dctresult[orig_name],
        "options":[]
    }
    for o in options:
        if o["name"] == item["name"]:
            similars_items[name]["options"].append(o)


In [39]:
# name_to_kw
result = {}

for name in tqdm(names):
    nums, orig_name = clear_name(name)  
    nums = set([int(n) for n in nums])
    names_list = [o["url"] for o in dctresult[orig_name]["options"]]
    
    scores = []
    scores_wordsim = []
    scores_num = []    
    
    for o in options:
        simwords = len(name_to_kw[name]&name_to_kw[o["orig"]])
        onum = [int(n) for n in o["nums"]]
        option_names_list = [opt["url"] for opt in o["options"]]
        
        scores_num.append(len(nums&set(onum)))
        if names_list and option_names_list:
            scores_wordsim.append(2*len(set(names_list)&set(option_names_list)) + 10*(names_list[0]==option_names_list[0]))
        else:
            scores_wordsim.append(0)
        scores.append(simwords)
    
    maxw = max(scores_wordsim)
    maxs = max(scores)
    
    for i in range(len(options)):
        if scores_wordsim[i] == maxw:
            scores_wordsim[i] *= (scores_num[i] + 1)
        else:
            scores_wordsim[i] = 0
            
        if scores[i] == maxs:
            scores[i] *= (scores_num[i] + 1)
        else:
            scores[i] = 0
            
    maxw = max(scores_wordsim)
    maxs = max(scores)
    
    result[name]={
        "maxw":maxw,
        "maxs":maxs,
        "scores_wordsim_res":options[scores_wordsim.index(maxw)],
        "scores_res":options[scores.index(maxs)],
        
    }

/home/neafiol/Documents/Projects/YaMarket/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [13]:
def get_params(spec):
#     print(spec)
    return {
        "type":spec.get("Тип",""),
        "fatness":spec.get("Жирность",""),
        "sort":spec.get("Сорт",""),
        "fasofka":spec.get("Фасовка",""),
        "milk_type":spec.get("Тип молока",""),
        "cheese_type":spec.get("Вид сыра",""),
        "weight":spec.get("Вес",""),
        
    }

In [3]:
# with open("result.pk","rb") as f:
#     pickle.dump(result,f)
    
with open("result.pk","rb") as f:
    result = pickle.load(f)

In [54]:
ans = []
for r in names:
    item = result[r]
    nums, orig_name = clear_name(r)
    
    if item["maxw"]:
        ans.append({
            "catalog_name":r,
            "item_name":item["scores_wordsim_res"]["orig"],
            "score":item["maxw"],
            **get_params(dctresult[orig_name]["specifications"])
            
        })
    else:
        ans.append({
            "catalog_name":r,
            "item_name":item["scores_res"]["orig"],
            "score":item["maxs"],
             **get_params(dctresult[orig_name]["specifications"]),
        })
        
    if item["maxw"] < 10:
        ans[-1]["mark_type"] = 3
    elif item["maxw"] > 20:
        ans[-1]["mark_type"] = 1
    elif item["maxw"] >= 10:
        ans[-1]["mark_type"] = 2
        
    if item["maxw"] < 5:
        ans[-1]["mark_type"] = 4
        
    ans[-1]["yamarket"] = dctresult[orig_name]["url"]
        
    

In [55]:
ans[100]

{'catalog_name': 'СЫР СВЕЖИЙ СЫР С ПЛЕСЕНЬЮ РОКФОРТИ ROYAL CHEESE 0,1 кг',
 'item_name': 'LEADER CHEESE СЫР КОРОЛЕВСКИЙ 1 кг.',
 'score': 12,
 'type': 'сыр',
 'fatness': '60 %',
 'sort': 'роял чиз',
 'fasofka': 'кусок',
 'milk_type': '',
 'cheese_type': 'мягкий',
 'weight': '',
 'mark_type': 2,
 'yamarket': 'https://market.yandex.ru/product--syr-kaloriia-royal-cheese-korolevskii-syr-s-goluboi-pleseniu-60/660133010'}

In [56]:
sum([a["score"] > 10 for a in ans])

36077

In [57]:
len(ans)

49436

In [72]:
n = len(ans)
for i in tqdm(range(0,n,500)):
    MergeData.insert_many(ans[i:i+500]).on_conflict_ignore().execute()

In [73]:
df = pd.DataFrame.from_dict(ans)

In [74]:
df.to_csv("res.csv")

In [75]:
df

,catalog_name,cheese_type,fasofka,fatness,item_name,mark_type,milk_type,score,sort,type,weight,yamarket
0,"СЫР ТВЕРДЫЙ БРУСОК КАЧОТТА FORMAGGI 0,12 кг тр...",мягкий,сырная голова,39 %,FORMAGGI СЫР КАЧОТТА СЛИВОЧНЫЙ 1 УПАКОВКА 120 г.,4,,4,качотта,сыр,,https://market.yandex.ru/product--syr-don-pepp...
1,"СЫР ТВЕРДЫЙ БРУСОК КОЗИЙ KOZARI 0,27 кг 45%",твердый,сырная голова,45 %,БАХЕТЛЕ СЫР КОЗИЙ КАБРА ТВЕРДЫЙ 100 г.,2,козье,15,,сыр,,https://market.yandex.ru/product--syr-karlov-d...
2,СЫР ПРОЧИЙ СЫР ПРОЧИЙ 1 кг 50%,мягкий,,60 %,БАХЕТЛЕ СЫР ИТАЛЬЯНСКИЙ 1 кг.,1,,24,,сыр,,https://market.yandex.ru/product--syr-pinar-la...
3,СЫР ТВЕРДЫЙ ВЕСОВОЙ РОССИЙСКИЙ ИЧАЛКИ 1 кг 45%,твердый,кусок,36 %,ИЧАЛКИ СЫР КАЛЬВЕТ 1 кг.,1,коровье,24,пармезан,сыр,,https://market.yandex.ru/product--syr-ichalki-...
4,СЫР ТВЕРДЫЙ СЛАЙСЕРНАЯ НАРЕЗКА ЛЕГКИЙ АЛАНТАЛЬ...,полутвердый,нарезка,35 %,АЛАНТАЛЬ СЫР ДЕЛИКАТЕСНЫЙ 1 УПАКОВКА 125 г.,3,коровье,6,,сыр,,https://market.yandex.ru/product--syr-alantal-...
5,"СЫР ПЛАВЛЕНЫЙ ЛОМТЕВОЙ БУРЕНКИН ЛУГ 0,1 кг 30%...",плавленый,порционная,30 %,БУРЕНКИН ЛУГ ПРОДУКТ ПЛАВЛЕНЫЙ С СЫРОМ КОЛБАСН...,2,коровье,10,,сырный продукт,,https://market.yandex.ru/product--syrnyi-produ...
6,"СЫР СВЕЖИЙ ФЕТА СИРТАКИ 0,2 кг 55% контейнер",рассольный,,60 %,HOCHLAND ПРОДУКТ СЫРНЫЙ ПЛАВЛЕНЫЙ ФЕТАКСА 1 У...,1,коровье,22,фета,сырный продукт,,https://market.yandex.ru/product--syrnyi-produ...
7,СЫР ПЛАВЛЕНЫЙ КОЛБАСНЫЙ ГОРОД СЫРА 1 кг 40% ВЕ...,плавленый,,40 %,ГОРОД СЫРА СЫР ПЛАВЛЕНЫЙ КОПЧЕНЫЙ КОЛБАСНЫЙ 1 кг.,1,коровье,32,,сыр,,https://market.yandex.ru/product--syr-gorod-sy...
8,СЫР ТВЕРДЫЙ ВЕСОВОЙ РОССИЙСКИЙ ДОБРЯНА 1 кг 50%,полутвердый,сырная голова,50 %,СЫР-БОР ПРОДУКТ ПОЛУТВЕРДЫЙ РОССИЙСКИЙ 1 кг.,1,коровье,22,российский,сыр,,https://market.yandex.ru/product--syr-dobriana...
9,СЫР ТВЕРДЫЙ ВЕСОВОЙ БУКОВИНСКИЙ СИБИРСКИЙ ГОСТ...,,сырная голова,45 %,НЕИДЕНТИФИЦИРУЕМАЯ МАРКА СЫР БУКОВИНСКИЙ 1 кг.,1,,22,буковинский,сыр,,https://market.yandex.ru/product--syr-syrnaia-...


# Test

In [ ]:
# test["UniqueName"] 
merged = data2.merge(test, left_on='Полное наименование товара', right_on='Name')

In [ ]:
merged

In [ ]:
merged = test.to_dict("records")
result = {m["UniqueName"]:m["Name"] for m in merged}

In [ ]:
NUM = r"\d+[,\.]?\d*"

def to_float(string):
    return float(re.search(NUM,string).group(0).replace(",","."))

def to_set(name):
    data = []
    p = re.search(f"{NUM}\s*кг",name)
    if p:
        data.append(to_float(p.group(0))*1000)
        name = name.replace(p.group(0),"")
    
    p = re.search(f"{NUM}\s*г",name)
    if p:
        data.append(to_float(p.group(0)))
        name = name.replace(p.group(0),"")
        
    p = re.search(f"{NUM}\s*%",name)
    if p:
        data.append(to_float(p.group(0)))
        name = name.replace(p.group(0),"")
        
    name = re.sub("[^а-яa-z\-]"," ",name.lower())
    name = re.sub("\s+"," ",name.lower())
    return data , name

In [ ]:
to_set("АЛЕКСАНДРОВСКАЯ ФЕРМА МАСЛО  СЛИВОЧНОЕ 72.5% 1 УПАКОВКА 180 г. ")

In [ ]:
data = data.to_dict('records')
data2 = data2.to_dict('records')

In [ ]:
all_items = []
my_data = []

for d in tqdm(data):
    all_items.append({
        "kw": to_set(" ".join([v for v in d.values() if isinstance(v,str)])),
        "kw_num": to_set(" ".join([v for v in d.values() if isinstance(v,float)])),
        "UniqueName" : d["UniqueName"]
    })

for d in tqdm(data2):
    my_data.append({
        "kw": to_set(" ".join([v for v in d.values() if isinstance(v,str)])),
        "kw_num": to_set(" ".join([v for v in d.values() if isinstance(v,float)])),
        "UniqueName" : d['Полное наименование товара']
    })
len(data)

In [ ]:
for d in tqdm(all_items):
    max_sim = 0
    options = []
    for i, o in enumerate(my_data):
        n = len(o['kw']&d["kw"])
        if n > max_sim:
            max_sim = n
            options = [i]
        if n == max_sim:
            options.append(i)    
            
    d["options"] = options

In [ ]:
keywords = []
for d in my_data:
    if "кезский" in d["kw"] and "сыр" in d["kw"]:
        print(d)
        break

In [ ]:
# result

In [ ]:
for o in all_items:
    if o["UniqueName"] in result:
        o["find"] = False
        items = []
        for var in o["options"]:
            if my_data[var]["UniqueName"] == result[o["UniqueName"]]:
                o["find"] = True
                break
            items.append(my_data[var]["UniqueName"])
        o["items"] = set(items)

In [ ]:
sum(["find" in d and d["find"] and len(d["items"]) < 3 for d in all_items])

In [ ]:
4992/(5526+2338)

In [ ]:
len(all_items)

In [ ]:
# for m in merged:
#     print(m['Полное наименование товара'],"-->",m["UniqueName"])

In [ ]:
mdat = pd.DataFrame.from_dict(my_data)

In [ ]:
mdat

In [ ]:
data.loc[:,data.columns.isin(["Name","UniqueName"])].head()

In [ ]:
data2['Полное наименование товара'][20]

In [ ]:
"МАСЛО СЛИВОЧНОЕ КРЕСТЬЯНСКОЕ 72.5%".lower()

In [ ]:
data2.columns

In [ ]:
data2.loc[:,data2.columns.isin(['Полное наименование товара','Наименование товара',"Бренд"])]

In [ ]:
set(data2["Категория"])

In [ ]:
names= list(data[data["SubCategoryName"].apply(lambda x:"СЫР" in x)]["UniqueName"])

In [ ]:
names

In [ ]:
set(data2["Уровень дерева категорий 5"])

In [ ]:
names2 = list(data2["Полное наименование товара"])

In [ ]:
def f(names2):
    names2 = [re.sub(r"[^a-zа-я ]","",w.lower()) for w in names2]
    names2 = [w.split("упаковка")[0].replace("кг","").replace("г","").rstrip() for w in names2]
    return names2

In [ ]:
for n in names2:
    words = set(n.split(" "))
    for w in names:
        if len(words & set(w.split(" "))) > 2:
            print(w)

In [ ]:
f(names)

In [ ]:
names